# UTILS

In [ ]:
import pandas as pd
import re
import string
import pickle
from sklearn.model_selection import train_test_split, GridSearchCV, ShuffleSplit
from sklearn.ensemble import AdaBoostClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, accuracy_score, recall_score, f1_score, precision_score
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC
from xgboost import XGBClassifier

In [ ]:
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W", " ", text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

## BOTH

### Pre-processing

### ISOT

In [ ]:
df_fake = pd.read_csv("./ISOT/Fake.csv")
df_true = pd.read_csv("./ISOT/True.csv")

df_fake["class"] = 0
df_true["class"] = 1

df_merge = pd.concat([df_fake, df_true], axis=0)
df_isot = df_merge.drop(["title", "subject", "date"], axis=1)
df_isot = df_isot.sample(frac=1)
df_isot.reset_index(inplace=True)
df_isot.drop(["index"], axis=1, inplace=True)

print(df_isot)

In [ ]:
df_isot["text"] = df_isot["text"].apply(wordopt)

print(df_isot)

In [ ]:
X_isot = df_isot["text"]
y_isot = df_isot["class"]

print(y_isot)

### LIAR

In [ ]:
column_names = [
    'id', 'label', 'statement', 'subjects', 'speaker', 'job_title', 'state_info', 
    'party_affiliation', 'barely_true_counts', 'false_counts', 'half_true_counts', 
    'mostly_true_counts', 'pants_on_fire_counts', 'context'
]

In [ ]:
df_test = pd.read_csv('./LIAR/test.tsv', sep='\t', names=column_names)
df_train = pd.read_csv('./LIAR/train.tsv', sep='\t', names=column_names)

In [ ]:
df_liar = pd.concat([df_train[['label', 'statement']], df_test[['label', 'statement']]], ignore_index=True)
df_liar

In [ ]:
class_mapping = {
    'true': 1,
    'mostly-true': 1,
    'half-true': 1,
    'barely-true': 0,
    'false': 0,
    'pants-fire': 0
}

In [ ]:
df_liar['class'] = df_liar['label'].map(class_mapping)
df_liar

In [ ]:
df_liar["statement"] = df_liar["statement"].apply(wordopt)
df_liar

In [ ]:
X_liar = df_liar["statement"]
y_liar = df_liar["class"]

print(y_liar)

In [ ]:
X = pd.concat([X_isot, X_liar], axis=0)
y = pd.concat([y_isot, y_liar], axis=0)

vectorizer = TfidfVectorizer()
X_vect = vectorizer.fit_transform(X)

X_vect.shape

In [ ]:
print(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_vect, y, test_size=0.2, random_state=42)

print(X_train.shape)
print(X_test.shape)  

## Base Learners

### Decision Tree

In [ ]:
param_grid_dt = {
    'max_depth': [None, 10],
    'min_samples_split': [2, 10],
    'min_samples_leaf': [1, 2]
}

In [ ]:
dt = DecisionTreeClassifier()

In [ ]:
grid_search_dt = GridSearchCV(estimator=dt, param_grid=param_grid_dt, cv=5, n_jobs=-1, verbose=2)

In [ ]:
grid_search_dt.fit(X_train, y_train)

In [ ]:
with open('grid_search_dt.pkl', 'wb') as file:
    pickle.dump(grid_search_dt, file)

In [ ]:
print("Melhor score de validação:", grid_search_dt.best_score_)

### Logistic Regression

In [ ]:
param_grid_lr = {
    'C': [0.1, 1, 10],
    'penalty': ['l2'],
    'solver': ['lbfgs'],
    'max_iter': [100, 200]
}

In [ ]:
lr = LogisticRegression()

In [ ]:
grid_search_lr = GridSearchCV(estimator=lr, param_grid=param_grid_lr, cv=5, n_jobs=-1, verbose=2)

In [ ]:
grid_search_lr.fit(X_train, y_train)

In [ ]:
with open('grid_search_lr.pkl', 'wb') as file:
    pickle.dump(grid_search_lr, file)

In [ ]:
print("Melhor score de validação:", grid_search_lr.best_score_)

### Naive Bayes

In [ ]:
param_grid_nb = {
    'alpha': [0.1, 0.5, 1.0, 1.5],
    'fit_prior': [True, False]     
}

In [ ]:
nb = MultinomialNB()

In [ ]:
grid_search_nb = GridSearchCV(estimator=nb, param_grid=param_grid_nb, cv=5, n_jobs=-1, verbose=2)

In [ ]:
grid_search_nb.fit(X_train, y_train)

In [ ]:
with open('grid_search_nb.pkl', 'wb') as model_file:
    pickle.dump(grid_search_nb, model_file)

In [ ]:
print("Melhor score de validação:", grid_search_nb.best_score_)

### SVM

In [ ]:
param_grid_svm = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto'],
    'class_weight': [None, 'balanced']
}

In [ ]:
svm = SVC(random_state=42)

In [ ]:
grid_search_svm = GridSearchCV(estimator=svm, param_grid=param_grid_svm, cv=5, n_jobs=-1, verbose=2)

In [ ]:
grid_search_svm.fit(X_train, y_train)

In [ ]:
with open('grid_search_svm.pkl', 'wb') as model_file:
    pickle.dump(grid_search_svm, model_file)

In [ ]:
print("Melhor score de validação:", grid_search_svm.best_score_)

## Bagging

### Random Forest

In [ ]:
param_grid_rf = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10],
    'min_samples_split': [2, 10]
}


In [ ]:
rf = RandomForestClassifier(random_state=42)

In [ ]:
grid_search_rf = GridSearchCV(estimator=rf, param_grid=param_grid_rf, cv=5, n_jobs=-1, verbose=2)

In [ ]:
grid_search_rf.fit(X_train, y_train)

In [ ]:
with open('grid_search_rf.pkl', 'wb') as file:
    pickle.dump(grid_search_rf, file)

In [ ]:
print("Melhor score de validação:", grid_search_rf.best_score_)

## Boosting

### AdaBoost

In [ ]:
param_grid_ada = {
    'n_estimators': [50, 100, 200], 
    'learning_rate': [0.01, 0.1, 0.5, 1],
    'algorithm': ['SAMME', 'SAMME.R'], 
}

In [ ]:
ada = AdaBoostClassifier(random_state=42)

In [ ]:
grid_search_ada = GridSearchCV(estimator=ada, param_grid=param_grid_ada, cv=5, n_jobs=-1, verbose=2)

In [ ]:
grid_search_ada.fit(X_train, y_train)

In [ ]:
with open('grid_search_ada.pkl', 'wb') as f:
    pickle.dump(grid_search_ada, f)

In [ ]:
print("Melhor score de validação:", grid_search_ada.best_score_)

### XGBoost

In [ ]:
param_grid_xgb = {
    'n_estimators': [100, 200],
    'learning_rate': [0.1, 0.3],
    'max_depth': [3, 5],
    'subsample': [0.8, 1.0],
    'gamma': [0, 0.1]
}


In [ ]:
xgb = XGBClassifier(random_state=42)

In [ ]:
grid_search_xgb = GridSearchCV(estimator=xgb, param_grid=param_grid_xgb, cv=5, n_jobs=-1, verbose=2)

In [ ]:
grid_search_xgb.fit(X_train, y_train)

In [ ]:
with open('grid_search_xgb.pkl', 'wb') as f:
    pickle.dump(grid_search_xgb, f)

In [ ]:
print("Melhor score de validação:", grid_search_xgb.best_score_)

## Stacking

In [ ]:
param_grid_stacking = {
    'final_estimator__C': [0.01, 0.1, 1.0, 10.0],  
    'final_estimator__solver': ['liblinear', 'saga'],  
    'passthrough': [False, True]  
}

In [ ]:
base_estimators = [
    ('lr', grid_search_lr.best_estimator__),
    ('nb', grid_search_nb.best_estimator__,
    ('dt', grid_search_dt.best_estimator__)
    ('svm', grid_search_svm.best_estimator__)
]

In [ ]:
meta_estimator = LogisticRegression(max_iter=1000, random_state=42)

In [ ]:
stacking = StackingClassifier(estimators=base_estimators, final_estimator=meta_estimator)

In [ ]:
grid_search_stacking = GridSearchCV(estimator=stacking, param_grid=param_grid_stacking, cv=5, verbose=1, n_jobs=-1)

In [ ]:
grid_search_stacking.fit(X_train, y_train)

In [ ]:
with open('grid_search_stacking.pkl', 'wb') as f:
    pickle.dump(grid_search_stacking, f)

In [ ]:
print("Melhor score de validação:", grid_search_stacking.best_score_)

# RESULTS

In [5]:
import pickle
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load the saved grid search objects
with open('./BD2_CV_RESULTS/grid_search_dt.pkl', 'rb') as f:
    grid_search_dt = pickle.load(f)

with open('./BD2_CV_RESULTS/grid_search_nb.pkl', 'rb') as f:
    grid_search_nb = pickle.load(f)

with open('./BD2_CV_RESULTS/grid_search_lr.pkl', 'rb') as f:
    grid_search_lr = pickle.load(f)

with open('.BD2_CV_RESULTS/grid_search_linear_svm.pkl', 'rb') as f:
    grid_search_svm = pickle.load(f)


# Extract the best models from the grid search
best_dt = grid_search_dt.best_estimator_
best_nb = grid_search_nb.best_estimator_
best_lr = grid_search_lr.best_estimator_
best_rf = grid_search_rf.best_estimator_
#best_ada = grid_search_ada.best_estimator_


# Define a function to compute and print metrics
def print_metrics(model, X_test, y_test, model_name):
    y_pred = model.predict(X_test)
    
    print(f"Metrics for {model_name}:")
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.9f}")
    print(f"Precision: {precision_score(y_test, y_pred, average='weighted'):.9f}")
    print(f"Recall: {recall_score(y_test, y_pred, average='weighted'):.9f}")
    print(f"F1-Score: {f1_score(y_test, y_pred, average='weighted'):.9f}")
    print("-" * 40)

# Assuming you have the test data X_test, y_test
# Replace these with your actual test data
# X_test, y_test = ...

# Print metrics for each model
print_metrics(best_dt, X_test, y_test, 'Decision Tree')
print_metrics(best_nb, X_test, y_test, 'Naive Bayes')
print_metrics(best_lr, X_test, y_test, 'Logistic Regression')
print_metrics(best_rf, X_test, y_test, 'Random Forest')
#print_metrics(best_ada, X_test, y_test, 'AdaBoost')

FileNotFoundError: [Errno 2] No such file or directory: '.BD2_CV_RESULTS/grid_search_linear_svm.pkl'

In [2]:
import numpy as np
x_values = np.linspace(0, 7, 100)
prob = clf.predict_proba(x_values.reshape(-1, 1))[:, 1]

plt.plot(x_values, prob, label="Sigmoid")
plt.scatter(X, y, color="red", label="Data Points")
plt.xlabel("Feature Value")
plt.ylabel("Probability")
plt.title("Regressão Logística: Curva Sigmoide")
plt.legend()
plt.show()

NameError: name 'clf' is not defined